In [6]:
# !pip install torch

In [7]:
import soundfile as sf
import torch
import torchaudio
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration

In [8]:
# def transcribe_audio_huggingface1(audio_file):
#     # Load the pre-trained Speech2Text model and processor
#     processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
#     model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
    
#     # Load and preprocess the audio file
#     speech_array, sampling_rate = torchaudio.load(audio_file)
#     speech = speech_array.squeeze().numpy()

#     # Ensure the sample rate matches the model's requirements
#     if sampling_rate != 16000:
#         resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
#         speech_array = resampler(speech_array)

#     # Process the audio data
#     inputs = processor(speech_array.squeeze(), sampling_rate=16000, return_tensors="pt")
    
#     # Generate transcription (speech-to-text)
#     with torch.no_grad():
#         generated_ids = model.generate(inputs["input_features"])
#         transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

#     print("Transcription:", transcription)
#     return transcription

In [9]:
# Call the function with your audio file path (wav or mp3 file)
# transcribe_audio_huggingface('audio_files\S2T_test_audio.wav')

In [14]:
def transcribe_audio_huggingface(audio_file):
    # Load the pre-trained Speech2Text model and processor
    processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
    model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
    
    # Set device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Load and preprocess the audio file using soundfile (for WAV files)
    speech_array, sampling_rate = sf.read(audio_file)

    # Ensure the sample rate matches the model's requirements
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(torch.tensor(speech_array, dtype=torch.float32).unsqueeze(0)).squeeze().numpy()

    speech_array = speech_array.astype('float32')

    # Process the audio data
    inputs = processor(speech_array, sampling_rate=16000, return_tensors="pt")
    
    # Move inputs to the same device as the model
    input_features = inputs["input_features"].to(device).float()

    # Generate transcription (speech-to-text)
    with torch.no_grad():
        generated_ids = model.generate(input_features)
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print("Transcription:", transcription)
    return transcription

In [15]:
# Call the function with your audio file path (WAV file)
transcribe_audio_huggingface("audio_files/S2T_test_audio.wav")

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Transcription: ah oh mamma came on ah they shall go back i shall go back i'll put a pullet down i'll put it back back back back back back back back and back back back back back back and back back back back back and back back back back back and back back back back and back back back back and back back back and back and back back back and back back back back and back and back and back and back back back and back back and back and back and back back back back back back back and back back back back back back back back back back back back back back back and back back back back back back back back back back back back and back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back


"ah oh mamma came on ah they shall go back i shall go back i'll put a pullet down i'll put it back back back back back back back back and back back back back back back and back back back back back and back back back back back and back back back back and back back back back and back back back and back and back back back and back back back back and back and back and back and back back back and back back and back and back and back back back back back back back and back back back back back back back back back back back back back back back and back back back back back back back back back back back back and back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back back"